In [ ]:
from termcolor import colored

import math as m
import numpy as np

import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.patches import Rectangle

In [ ]:
EXT_SIZE_X, EXT_SIZE_Y = 80, 60

INT_OFFS_X, INT_OFFS_Y = 20, 20
INT_SIZE_X, INT_SIZE_Y = 20, 20

EXT_BORDER = +300.0
INT_BORDER = +320.0

STEP = 1

def func(x, z):
    # return 30
    f0, beta = 60, 0.1
    x0, z0 = 45, 9
    return f0 * m.exp(-beta * ((x - x0)**2 + (z - z0)**2))

K = 20

In [ ]:
STEPS_1X = int(INT_OFFS_X / STEP)
STEPS_2X = int((INT_SIZE_X + 1) / STEP)
STEPS_3X = EXT_SIZE_X - INT_OFFS_X - INT_SIZE_X

STEPS_1Y = int(INT_OFFS_Y / STEP)
STEPS_2Y = int((INT_SIZE_Y + 1) / STEP)
STEPS_3Y = EXT_SIZE_Y - INT_OFFS_Y - INT_SIZE_Y

STEPS_X = STEPS_1X + STEPS_2X + STEPS_3X
STEPS_Y = STEPS_1Y + STEPS_2Y + STEPS_3Y

In [ ]:
# Matrix and accessors

matrix = np.zeros((STEPS_Y, STEPS_X), dtype=float)
h, w = matrix.shape

ibx0, iby0 = INT_OFFS_X, INT_OFFS_Y
ibx1, iby1 = INT_OFFS_X + INT_SIZE_X, INT_OFFS_Y + INT_SIZE_Y

def is_ext_bound(y, x):
    return x in [0, w - 1] or y in [0, h - 1]

def _is_hole_or_bound(y, x):
    return x >= ibx0 and x <= ibx1 and y >= iby0 and y <= iby1

def _is_int_bound_1dim(y, x):
    return x in [ibx0, ibx1] or y in [iby0, iby1]

def is_int_bound(y, x):
    return _is_hole_or_bound(y, x) and _is_int_bound_1dim(y, x)

def is_hole(y, x):
    return _is_hole_or_bound(y, x) and not _is_int_bound_1dim(y, x)

def is_target(y, x):
    return not (_is_hole_or_bound(y, x) or is_ext_bound(y, x))


def get_range1x():
    return range(1, ibx0)

def get_range2x():
    return range(ibx0, ibx1 + 1)

def get_range3x():
    return range(ibx1 + 1, w - 1)

def get_range1y():
    return range(1, iby0)

def get_range2y():
    return range(iby0, iby1 + 1)

def get_range3y():
    return range(iby1 + 1, h - 1)

In [ ]:
# Bounds setup

def set_bounds(mat):
    for j in range(h):
        mat[j, 0] = EXT_BORDER
        mat[j, w - 1] = EXT_BORDER
    for i in range(w):
        mat[0, i] = EXT_BORDER
        mat[h - 1, i] = EXT_BORDER

    for y in get_range2y():
        mat[y, ibx0] = INT_BORDER
        mat[y, ibx1] = INT_BORDER
    for x in get_range2x():
        mat[iby0, x] = INT_BORDER
        mat[iby1, x] = INT_BORDER

set_bounds(matrix)

In [ ]:
# Creating first iteration by linear interpolation of bounds

def lerp(x, y, t):
    return (1 - t) * x + t * y

# Short intervals

for x in get_range2x():
    for y in get_range1y():
        matrix[y, x] = lerp(matrix[0, x], matrix[iby0, x], y / STEPS_1Y)
    for y in get_range3y():
        matrix[y, x] = lerp(matrix[iby1, x], matrix[h - 1, x], (y - iby1) / STEPS_3Y)

for y in get_range2y():
    for x in get_range1x():
        matrix[y, x] = lerp(matrix[y, 0], matrix[y, ibx0], x / STEPS_1X)
    for x in get_range3x():
        matrix[y, x] = lerp(matrix[y, ibx1], matrix[y, w - 1], (x - ibx1) / STEPS_3X)

# Corners

for x in get_range1x():
    for y in get_range1y():
        matrix[y, x] = lerp(matrix[0, x], matrix[iby0, x], y / STEPS_1Y)
for x in get_range1x():
    for y in get_range3y():
        matrix[y, x] = lerp(matrix[iby1, x], matrix[h - 1, x], (y - iby1) / STEPS_3Y)
for x in get_range3x():
    for y in get_range1y():
        matrix[y, x] = lerp(matrix[0, x], matrix[iby0, x], y / STEPS_1Y)
for x in get_range3x():
    for y in get_range3y():
        matrix[y, x] = lerp(matrix[iby1, x], matrix[h - 1, x], (y - iby1) / STEPS_3Y)

# for j in range(1, h - 1):
#     for i in range(1, w - 1):
#         matrix[j, i] = INT_BORDER

$
\dfrac{\partial^2 u}{\partial x^2} + \dfrac{\partial^2 u}{\partial z^2} = F(x, z) = - \dfrac{f(x, z)}{k}
$

$
\dfrac{1}{h_x^2}(u_{i+1,j} - 2u_{i,j} + u_{i-1,j}) + \dfrac{1}{h_z^2}(u_{i,j+1} - 2u_{i,j} + u_{i-1,j-1}) = F(x, z)
$

Примем $h_x = h_z = h$:

$
u_{i+1,j} - 2u_{i,j} + u_{i-1,j} + u_{i,j+1} - 2u_{i,j} + u_{i-1,j-1} = h^2 F(x, z)
$

$
u_{i,j} = \frac{1}{4}(u_{i+1,j} + u_{i-1,j} + u_{i,j+1} + u_{i-1,j-1} - h^2 F(x_i, z_j))
$

Условие окончания счёта: $\|u^{(k+1)} - u^{(k)}\| < \varepsilon$, где $\|u\| = \max_{i,j}{(u_{i,j})}$

In [ ]:
# Zeidel

MAX_ITER = 3000
EPS = 1e-2

def func2(x, z): return -func(x, z) / K

func2_values = np.zeros(matrix.shape, dtype=float)
# set_bounds(func2_values)
for j in range(1, h - 1):
    for i in range(1, w - 1):
        func2_values[j, i] = func2(j * STEP, i * STEP)

# Returns true if solution is found
def single_iter():
    iter_max = -m.inf
    step2 = STEP * STEP
    for j in range(1, h - 1):
        for i in range(1, w - 1):
            if not is_target(j, i):
                continue
            val = 0.25 * (
                matrix[j, i + 1] +
                matrix[j, i - 1] +
                matrix[j + 1, i] +
                matrix[j - 1, i] -
                step2 * func2_values[j, i]
            )
            diff = abs(val - matrix[j, i])
            iter_max = max(iter_max, diff)
            matrix[j, i] = val
    return iter_max < EPS

fin_iters = False
for iter in range(MAX_ITER):
    is_end = single_iter()
    if is_end:
        print(f'Iterations: {iter}')
        fin_iters = True
        break

if not fin_iters:
    print(f'Iterations: {MAX_ITER} (limit exceed)')

In [ ]:
# Heatmap
%matplotlib widget

vmin = min(min(x for x in row if x > 1e-3) for row in matrix)
plt.imshow(matrix, cmap=cm.plasma, vmin=vmin)
hole_rect = Rectangle((INT_OFFS_Y, INT_OFFS_X), INT_SIZE_Y, INT_SIZE_X,
                        edgecolor='black', facecolor='black')
plt.gca().add_patch(hole_rect)
plt.colorbar()
plt.show()

In [ ]:
print(matrix[45, 9])

$
\dfrac{\partial}{\partial x}\left(k(x, z)\dfrac{\partial u}{\partial x}\right) +
\dfrac{\partial}{\partial z}\left(k(x, z)\dfrac{\partial u}{\partial z}\right)
$

$
div(k \nabla u) = k \Delta u + \nabla k \nabla u
$

$k \Delta u$ было рассмотрено выше. Пусть для $k(x, z)$ извезтны производные по $x$ и $z$.

$
\nabla k \nabla u = k'_x \dfrac{\partial u}{\partial x} + k'_z \dfrac{\partial u}{\partial z}
$

Разностная схема для первой производной:

$
\dfrac{\partial u}{\partial x} = \dfrac{u_{i+1,j} + u_{i-1,j}}{2h_x}
$

Итого:

$
\dfrac{1}{h_x^2}(u_{i+1,j} - 2u_{i,j} + u_{i-1,j}) + \dfrac{1}{h_z^2}(u_{i,j+1} - 2u_{i,j} + u_{i-1,j-1})
+
\dfrac{1}{2h_x}(u_{i+1,j} + u_{i-1,j}) + \dfrac{1}{2h_z}(u_{i,j+1} + u_{i,j-1})
$